In [36]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


In [63]:

df = pd.read_csv('./ml_task_data.csv')
df['CompletionDate'] = pd.to_datetime(df.CompletionDate)
df['CompletionDate'] = df['CompletionDate'].dt.strftime('%Y')

df['SpudDate'] = pd.to_datetime(df.SpudDate)
df['SpudDate'] = df['SpudDate'].dt.strftime('%Y')

df.dropna(how='all', subset=['SpudDate', 'CompletionDate'], inplace=True)

In [64]:

basin_compdate_grouped = df.groupby(["Basin", "CompletionDate"])
basin_grouped = df.groupby(["Basin"])
state_grouped = df.groupby(["State"])
spud_date_grouped = df.groupby(['SpudDate'])


In [65]:

for key, item in spud_date_grouped:
    df.loc[(df['SpudDate'] == key), 'CompletionDate'] = item['CompletionDate'].fillna(item['CompletionDate'].mode()[0])


In [66]:

for key, item in state_grouped:
    state = key

    # print(key)
    # current_group = df.loc[(df["Basin"] == basin) & (df["CompletionDate"] == compdate)]
    
    basin_mode = item["Basin"].mode()[0]
    # print(basin_mode)
    # Updating basin nan values
    df.loc[(df["State"] == state), "Basin"] = item["Basin"].fillna(basin_mode)
    

In [67]:
for key, item in basin_grouped:
    basin = key

    # print(key)
    # current_group = df.loc[(df["Basin"] == basin) & (df["CompletionDate"] == compdate)]
    
    state_mode = item["State"].mode()[0]
    # print(basin_mode)
    # Updating basin nan values
    df.loc[(df["Basin"] == basin), "State"] = item["State"].fillna(state_mode)

    df.loc[(df["Basin"] == basin), "Latitude"] = item["Latitude"].fillna(item["Latitude"].median())
    df.loc[(df["Basin"] == basin), "Longitude"] = item["Longitude"].fillna(item["Longitude"].median())



In [68]:

for key, item in basin_compdate_grouped:
    basin = key[0]
    compdate = key[1]

    # current_group = df.loc[(df["Basin"] == basin) & (df["CompletionDate"] == compdate)]
    bvhh_median = item["BVHH"].median()
    lat_length_in_miles_median = item['LateralLengthInMiles'].median()

    if math.isnan(bvhh_median):
        bvhh_median = df.loc[(df["Basin"] == basin)]["BVHH"].median()
    
    if math.isnan(bvhh_median):
        bvhh_median = df.loc[(df["CompletionDate"] == compdate)]["BVHH"].median()
    
    if math.isnan(bvhh_median):
        bvhh_median = df["BVHH"].median()

    if math.isnan(bvhh_median):
        bvhh_median = 1


    if math.isnan(lat_length_in_miles_median):
        lat_length_in_miles_median = df.loc[(df["Basin"] == basin)]["LateralLengthInMiles"].median()
    
    if math.isnan(lat_length_in_miles_median):
        lat_length_in_miles_median = df.loc[(df["CompletionDate"] == compdate)]["LateralLengthInMiles"].median()
    
    if math.isnan(lat_length_in_miles_median):
        lat_length_in_miles_median = df["LateralLengthInMiles"].median()

    if math.isnan(lat_length_in_miles_median):
        lat_length_in_miles_median = 0.75

    
    # Updating bvhh nan values
    df.loc[(df["Basin"] == basin) & (df["CompletionDate"] == compdate), "BVHH"] = item['BVHH'].fillna(bvhh_median)

    # Updating LateralLengthInMiles nan values
    df.loc[(df["Basin"] == basin) & (df["CompletionDate"] == compdate), "LateralLengthInMiles"] = item['LateralLengthInMiles'].fillna(lat_length_in_miles_median)
    


In [69]:
state_grouped = df.groupby(["State"])

for key, item in state_grouped:
    state = key
    
    formation_alias_mode_arr = item['formationAlias'].mode()
    
    if len(formation_alias_mode_arr) == 0: # check if mode is nan
        formation_alias_mode = df["formationAlias"].mode()

    df.loc[(df['State'] == state), 'formationAlias'] = item['formationAlias'].fillna(formation_alias_mode[0])


In [70]:
df.isnull().sum(axis = 0)

Basin                      5
State                      5
Latitude                   0
Longitude                  0
PadName                  402
SpudDate                 511
CompletionDate             0
OperatorAlias           2203
formationAlias             0
LateralLengthInMiles       0
BVHH                       1
proppantPerFoot            0
dtype: int64

In [73]:
# Features -> Latitude, formationAlias, BVHH
df.dropna(how='any', subset=['BVHH', 'formationAlias', 'Latitude'], inplace=True) # drop rows where any of the chosen features are nan

df.count()

Basin                   51464
State                   51464
Latitude                51468
Longitude               51468
PadName                 51066
SpudDate                50957
CompletionDate          51468
OperatorAlias           49265
formationAlias          51468
LateralLengthInMiles    51468
BVHH                    51468
proppantPerFoot         51468
dtype: int64